#                                  CIS600 - Social Media & Data Mining
#                                  Twitter Influential Users
###  
<img src="https://www.syracuse.edu/wp-content/themes/g6-carbon/img/syracuse-university-seal.svg?ver=6.3.9" style="width: 200px;"/>

### April 24, 2018

In [1]:
#Following are the Necessary libraries needed to run the code
import tweepy
import json
import sys
import jsonpickle
import os
import time
import csv
import pandas as pd
import json
from tweepy import OAuthHandler
from tweepy import Stream
from tweepy import StreamListener
from time import gmtime, strftime
from datetime import datetime
import re
import numpy as np
import matplotlib.pyplot as plt
import plotly.offline as py
import plotly.graph_objs as go
import plotly
import matplotlib.pyplot as plt; plt.rcdefaults()
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import bar
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from scipy.misc import imread
import matplotlib.pyplot as plt
import warnings
from PIL import Image

In [ ]:
#Authentication 
#Use auth_dict file for twitter  access
with open('auth_dict','r') as f:
    twtr_auth = json.load(f)

# Replace the API_KEY and API_SECRET with your application's key and secret.
#using app handler for 2.5 times faster results
auth = tweepy.AppAuthHandler(twtr_auth['consumer_key'], twtr_auth['consumer_secret'])

#API 
api =tweepy.API(auth, wait_on_rate_limit=True,wait_on_rate_limit_notify=True)

#checking connections
if (not api):
    print ("Can't Authenticate")

In [ ]:
retries=0

# this is the mention we ere looking for in twitter api
Query = '#WrestleMania'  
    
# number of tweets to fetch
maxTweets = 10000
    
# this is the max the API permits
tweetsPerQry = 10  
    
# We will store the tweets in a following json file in same directory.
fName = 'parent_Tweets_WrestleMania.json'

all_tweets = []
tweetCount =0
new_tweets = []
with open(fName, 'w') as f:
        
    #for tweet in tweepy.Cursor(api.search,q='#wrestlemania',result_type = 'popular').items(1):
    try:
    #used REST API for older tweets(one week older)
    #tweets for above query using cursor 
        data = tweepy.Cursor(api.search, q=Query,result_type = 'mixed').items(maxTweets)
         
    except tweepy.TweepError as e:
        print("Error when getting tweet: %s" % e)
        
    retries += 1
        
    if retries >= 1000:
        raise
    else:
        #put to sleep
        time.sleep(5)
    
    new_tweets = []
    
        #for checking the tweet data 
    for tweet in data:
        new_tweets.append(tweet)
        
    #saving tweets in a file
    for tweet in new_tweets:
        f.write(jsonpickle.encode(tweet._json, unpicklable=False) +'\n')
        
    tweetCount += len(new_tweets)
       
print ("Downloaded {0} tweets, Saved to {1}".format(tweetCount, fName))

In [ ]:
#Getting retweeted users of parent user who is tweeting on wrestlemania
def retweet_users_of_parent_tweet(tweet_id,screen_name):
    
    #Returns a collection of the 100 most recent retweets of the Tweet specified by the id parameter
    try:
        retweets = api.retweets(tweet_id, 100)
    
    #for removing the outliers
    except tweepy.TweepError as e:
        return None, e
    
    #returning id of retweeted user and screen-name of parent user 
    return [rt.user.id for rt in retweets],screen_name

In [ ]:
#Function to filter tweets of retweeted user based on hashtag
def tweet_user_all_tweets(user,n):
    
    #list of final tweets of a retweeted user
    result = []
    count = 0
    u_tweet=[]
        
    #including retweets and original tweets of this user(all tweets)    
    for tweet in tweepy.Cursor(api.user_timeline, id=user,include_rts=True).items(200):
        if re.findall(r'#WrestleMania', tweet._json['text'],re.I): 
            result.append(tweet._json)
    
    #returning tweets
    return result

In [ ]:
def tweet_user_mentions(user,screen_name):
    
    #getting the hasgtag based tweets of each retweeter
    tweets = tweet_user_all_tweets(user, 2)
    t_text = ''
        
    #extracting mentions here
    for t in tweets:
        t_text += t['text']
        
    # number of direct mentions + retweets
    #filtering on parent user 
    return len(re.findall(r"@%s" % screen_name, t_text, re.IGNORECASE))

In [ ]:
#Function to calculate score of each parent 
def tweet_user_score(users,screen_name):
    
    #dictionary to store features
    user_dic = {}
    count = 0
    
    for user in users:
        
        #getting the screen name of retweeted user
        retweeted_user_screen_name = api.get_user(id=user).screen_name
        
        #followers count of retweeted user
        follower = api.get_user(id=user).followers_count
                
        #getting mention of user
        mention = tweet_user_mentions(user,screen_name)
        
        #score calculation
        user_dic[retweeted_user_screen_name] = [follower, mention, (follower*mention)]
        
        
        count += 1
        
        print (count, 'of', len(users), 'users added into dictionary')
        
        if count%20 == 0:
            print ('sleep for one minute')
            time.sleep(60)
            
            
    return user_dic

In [ ]:
retries=0
count=0

for i in open('parent_Tweets_WrestleMania.json',"r"):
    if i=="\n":
        next
    #elif count <=500 :
        #count += 1
        #continue
    else:
        #handling twitter requests
        try:
            
            #processing of each tweet
            tweet = json.JSONDecoder().raw_decode(i)[0]
        
        except tweepy.TweepError as e:
            print("Error when getting tweet: %s" % e)
        retries += 1
        
        #giving the maximum retries
        if retries >= 10000:
            raise
        else:
            time.sleep(5)

    #getting the user screen name to further filter retweeters who retweet this user and tag
    screen_name=tweet['user']['screen_name']
    
    #getting user id of above extracted user
    tweet_id=tweet['id']
    
    #getting the retweeters of this parent user who talks about his parent and hastag in his tweets
    retweet_id, par_screenname = retweet_users_of_parent_tweet(tweet_id,screen_name)
    
    #check if no user was found
    if retweet_id is None:
        print ("Error in finding retweeter @{0}: {1}".format(screen_name, par_screenname))
        continue
    
    #calculation of Influential score  and stroing the results in dictionary
    user_dic =tweet_user_score(retweet_id, par_screenname)
    
    
    follower = [list(user_dic.values())[x][0] for x in range(len(user_dic))]
    mention = [list(user_dic.values())[x][1] for x in range(len(user_dic))]
    score = [list(user_dic.values())[x][2] for x in range(len(user_dic))]
    
    
    keys = []
    
    for i in user_dic.keys():
        keys.append(i)
        
    t_id = [tweet['id'] for x in range(len(user_dic))]
    
    
    newdic = {'tweet-id':tweet_id,'influencer':keys,'score':score,'mention':mention,'follower':follower,
             'parent-user':screen_name,'retweet_id':retweet_id,}
    
    count += 1
    
    #for keeping the track, how much of parents have been analyzed 
    print ('-------',count,'Tweets Analyzed', '-------')
    
    df = pd.DataFrame(newdic)
    
    #storing the results in csv file
    df.to_csv('WrestleMania_SuperFans/'+ screen_name + '.csv', encoding='utf-8')



In [ ]:
#Getting the csv's of SuperFans of WrestleMania
#since mac generated '.DS_Store files"

home_dir='WrestleMania_SuperFans/'

SuperFans=[]

#extract and sort files

for _, _,files in os.walk(home_dir):
    for file in files:  
        if file.endswith('.DS_Store'):
            continue
        SuperFans.append(file)
        SuperFans.sort(reverse=False)

# Results

Here we are stroring the results in the form of two csv files in the main home directory

1)Top 10 Parents of WrestleMania


2)Top 10 retweets of each parent 

In [ ]:
# Top 10 influencers of every parent user of WrestleMania
Top_Influencers=pd.DataFrame()

for f in SuperFans: #1000 parent users
    #Open file
    parent_influencers=pd.read_csv('WrestleMania_SuperFans/'+ f, 'r',delimiter=',')
    
    if parent_influencers.empty:
        
        continue



    top10=parent_influencers.nlargest(10, 'score')
    
    top10.rename(columns={'follower': 'Follower', 'influencer': 'Influencer','mention':'User_Mentions',
                         'parent-user':'Parent_User','Retweet_id':'Retweet_ID','score':'Influence_Score',
                         'tweet_id':'Parent_user_id'}, inplace=True)
    
    Top_Influencers=top10.append(Top_Influencers)
    
    
#storing the results in csv file
Top_Influencers.to_csv('Top10_influencers_WrestleMania.csv')

In [ ]:
Score=pd.DataFrame(columns=['User','Influence_Score'])

score_index=-1

for f in SuperFans: #1000 
    #Open file
    parent_influencers=pd.read_csv('WrestleMania_SuperFans/'+ f, 'r',delimiter=',')
    #print(parent_influencers['parent-user'])
    score_index +=1
    
    
    Score.set_value(score_index,'User',f[:-4])
    
    if parent_influencers.empty:
        Score.set_value(score_index,'Influence_Score', 0)
        continue
    else:
        total_score=parent_influencers['score'].sum()
        Score.set_value(score_index,'Influence_Score', total_score)
    
    #score_index +=1
    
Score.to_csv('Parent_influence_Score_WrestleMania.csv')



# Visualizations based on Results

Following the the functions for getting the graphs and other exploratory anlaysis results:

1)WordCloud of around 1000 Parent tweets 

2)Graph of Top 10 SuperFans 

3)Graph of Top 10 retweeters of each of the above Top 10 superFans


In [8]:
#Function to create word cloud in the form of twitter shape
def twitter_cloud(tweettag):
    tweet=[]
    for i in open('parent_Tweets_' + str(tweettag[1:]) + '.json',"r"):
        if i=="\n":
            next
        else:
            #handling exceptions
            tweet.append(json.JSONDecoder().raw_decode(i)[0])

    warnings.filterwarnings('ignore')
    text=[]
    
    for i in tweet:
        text.append(i['text'])
        

    res = "".join(text)
    no_urls_no_tags = " ".join([word for word in res.split() if 'http' not in word
                                    and not word.startswith('@')
                                    and word != 'RT'])

    #getting the twitter mask for the cloud
    mask = imread("./twitter_mask.png", flatten=True)

    wc = WordCloud(background_color="white", font_path="/Library/Fonts/Verdana.ttf", stopwords=STOPWORDS, width=7000,
                          height=800,mask=mask,colormap="Blues")
    wc.generate(no_urls_no_tags)
    
    #show cloud
    plt.imshow(wc)
    plt.axis("off")

    #saving the clud in main directory
    plt.savefig('twitter_clud.png', dpi=300)

In [ ]:
#Function showing the bar graph of Top 10 parent users and their score
def twitter_visualizations(tweettag):
    df1= pd.read_csv('Parent_influence_Score_' + str(tweettag[1:]) + '.csv')
    
    high_scorers=df1.nlargest(10, 'Influence_Score')
    high_scorers=high_scorers.drop(high_scorers.columns[high_scorers.columns.str.contains('unnamed',case = False)],axis = 1)
    Top_Users=list(high_scorers['User'])
    
    height = list(high_scorers['Influence_Score'])
    bars = list(high_scorers['User'])
    y_pos = np.arange(len(bars))

    # Create bars
    plt.bar(y_pos, height,width=0.8,color='blue',alpha=0.8)
    # Create names on the x-axis
    plt.xticks(y_pos, bars,rotation=90,color='black')
    plt.yticks(color='black')
    plt.xlabel('Twitter Users')
    plt.ylabel('Influence Scores')
    plt.grid(axis='y')
    #Show graphic
    
    plt.show()
    
    return Top_Users



In [ ]:
#function to. show the Graph of Top 10 retweeters of each of the above Top 10 superFans
def graph_based_on_parent_user(Top_Users,tweettag):
    
    df2= pd.read_csv('Top10_influencers_' + str(tweettag[1:]) + '.csv')
    df2=df2.drop(df2.columns[df2.columns.str.contains('unnamed',case = False)],axis = 1)
    #print(df2.head)
    for t in Top_Users: 
    
        
        
        d=df2.loc[df2['Parent_User']== t]
        height = list(d['Influence_Score'])
        bars = list(d['Influencer'])
        y_pos = np.arange(len(bars))

        # Create bars
        plt.bar(y_pos, height,width=0.8,color='blue',alpha=0.8)
        # Create names on the x-axis
        plt.xticks(y_pos, bars,rotation=90,color='black')
        plt.yticks(color='black')
        plt.xlabel('Top10 Twitter Users')
        plt.ylabel('Influence Scores')
        plt.grid(axis='y')
        #Show graphic
        
        print ("plotting Influence Scores of  " + str(t))
        
        plt.show()

Below are the steps to run visulaization Functions

In [ ]:
#function call to create file of Top 10 parent users
top_users=twitter_visualizations('#WrestleMania')

#function call to create file of top retweeters and their scores with respect to their parent users 
graph_based_on_parent_user(top_users,'#WrestleMania')

#function call to create word cloud
twitter_cloud('#WrestleMania')